In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [58]:
obs_data = pd.read_csv("observation_features.csv")


In [43]:
symptoms = obs_data.iloc[:,0:10]
age = obs_data.iloc[:,10]
gender = obs_data.iloc[:,11]
income = obs_data.iloc[:,12]
genome = obs_data.iloc[:,13:141]
comorbidities = obs_data.iloc[:,141:147]
vaccination_status = obs_data.iloc[:,147:]

In [44]:
vaccination_status

,0.0.76,0.0.77,0.0.78
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,0.0
...,...,...,...
99994,0.0,0.0,0.0
99995,1.0,0.0,0.0
99996,0.0,1.0,0.0
99997,1.0,0.0,0.0


You need to tune 'threshold' so that the method is reliable.
For example, you should define a probability with which you would like to include the correct feature and a probability with which you should not include useless features. It should return a list of feature indices
This is an example select_features implementation, but you should build your own, preferrably through an existing feature selection method. This implementation performs, separately for every feature, model selection versus the null hypothesis that Y is independent of X_i. That is, we have

$\mu_0: P(Y|X_i) = P(Y)$ 

$\mu_i: P(Y|X_i = 0) \neq P(Y)$.


This can be implemented through n Bayesian tests.

In [148]:
def select_features(X, Y, threshold):
    n_features = X.shape[1]
    n_data =  X.shape[0]
    alpha_b = np.ones([n_features, 2 ])
    beta_b = np.ones([n_features, 2])
    log_p = np.zeros(n_features)
    
    log_null = 0
    alpha = 1
    beta = 1  
    for t in range(n_data):
        p_null = alpha / (alpha + beta)
        log_null += np.log(p_null)*Y[t] + np.log(1-p_null)*(1 - Y[t])
        alpha += Y[t]
        beta += (1 - Y[t])
        for i in range(n_features):
            x_ti = X[t,i]
            p = alpha_b[i, x_ti] // (alpha_b[i, x_ti] + beta_b[i, x_ti])
            log_p[i] += np.log(p)*Y[t] + np.log(1-p)*(1 - Y[t])
            alpha_b[i, t] += Y[t]
            beta_b[i, t] += (1 - Y[t])
    p = np.exp(log_p) / (np.exp(log_p) + np.exp(log_null))

    features = []
    for i in range(n_features):
        if p[i] > threshold:
            features.append(i)
    return features

In [153]:
X = np.array(genome)
Y = np.array(symptoms)

In [161]:
select_features(X,Y[:,0],threshold= 0.80)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [159]:
X[1]

array([0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1.,
       0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0.,
       0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1.,
       1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1.,
       0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0.])